# **0. Load libraries**

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
#!pip install implicit
from implicit.als import AlternatingLeastSquares
from sklearn.metrics import precision_score, ndcg_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# og_file_path="~/code/Alanoudis/food-delivery-rec/data/updated_data/orders_sg25k.txt"
# og_data = pd.read_csv(og_file_path, sep=',', encoding='utf-8')
# og_data.head()

# og_products = "~/code/Alanoudis/food-delivery-rec/data/raw_data/products_sg.txt"
# og_products = pd.read_csv(og_products, sep=',', encoding='utf-8')
# og_products.head()

# og_vendors = "~/code/Alanoudis/food-delivery-rec/data/raw_data/vendors_sg.txt"
# og_vendors = pd.read_csv(og_vendors, sep=',', encoding='utf-8')
# og_vendors.head()

In [3]:
# old 100k data, im looking for products_rating and order_frequency columns
full_data100k_path = "~/code/Alanoudis/food-delivery-rec/data/test-train/full_data100k.csv"
full_data100k = pd.read_csv(full_data100k_path, index_col=0)
full_data100k.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,chain_id,vendor_geohash,cuisine_origin,order_frequency,product_rating
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,12:03:29,85 days,japanese garlic karaage don,6.0,66c9978d,w21z7,japanese,1,4
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,12:03:29,85 days,chicken cutlet don,6.8,66c9978d,w21z7,japanese,1,5
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,12:03:29,85 days,beef sukiyaki don,6.8,66c9978d,w21z7,japanese,1,3
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,12:03:29,85 days,japanese beef yakiniku don,6.8,66c9978d,w21z7,japanese,1,5
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,12:03:29,85 days,teriyaki salmon don,8.0,66c9978d,w21z7,japanese,1,5


In [4]:
full_data100k.shape

(100000, 15)

In [5]:
#full_data includes each product in each order + new columns for time features
full_data_path = "~/code/Alanoudis/food-delivery-rec/notebooks/alanoud/full_data_complete.csv"
full_data = pd.read_csv(full_data_path)
full_data.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour,meal_of_day,order_hour_sin,order_hour_cos
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,1900-01-01 12:03:29,85,japanese garlic karaage don,6.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,1900-01-01 12:03:29,85,chicken cutlet don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,1900-01-01 12:03:29,85,beef sukiyaki don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,1900-01-01 12:03:29,85,japanese beef yakiniku don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,1900-01-01 12:03:29,85,teriyaki salmon don,8.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0


In [6]:
full_data.columns

Index(['customer_id', 'customer_geohash', 'order_id', 'vendor_id',
       'product_id', 'day_of_week', 'order_time', 'order_day', 'name',
       'unit_price', 'chain_id', 'vendor_geohash', 'cuisine_origin',
       'is_weekend', 'dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5',
       'dow_6', 'order_hour', 'meal_of_day', 'order_hour_sin',
       'order_hour_cos'],
      dtype='object')

In [7]:
full_data.shape

(100000, 25)

In [8]:
#full_data2 includes 1 row per order. This is used for vendor ratings.
vendor_ratings_path = "~/code/Alanoudis/food-delivery-rec/notebooks/alanoud/feature-engineering/Alshaimas-rating/vendor_ratings.csv"
full_data2 = pd.read_csv(vendor_ratings_path)
full_data2 = full_data2.rename(columns={'avg_vendor_rating': 'vendor_rating'})
full_data2.head()

,customer_id,customer_geohash,order_id,vendor_id,day_of_week,order_time,order_day,chain_id,vendor_geohash,cuisine_origin,vendor_rating,num_products,total_order_value,products_ordered
0,00119c8178,w21zu,39095,e7cb5902,2,16:30:07,10 days,ef3142e8,w21zu,malaysian,3.5,1,5.6,Seafood Fried Rice
1,00198e01e4,w21z6,35939,02acaff6,0,13:57:35,85 days,8c51e46b,w21z6,chinese,3.9,4,8.0,"Bean Curd Skin 凉拌腐竹, Dried Beancurd 凉拌素鸡, Blac..."
2,001a5689fc,w21z3,48288,a2c60e71,0,20:10:35,78 days,dd69fe77,w21z3,singaporean,4.5,1,2.4,Carrot Cake (Melur)
3,001a5689fc,w21z3,48289,a3bc472c,4,20:50:40,40 days,c59edb7d,w21z9,snacks,4.0,1,1.6,Served without Fries
4,001a5689fc,w21z3,48290,de18b671,3,20:05:42,53 days,f88ffd2b,w21z3,american,4.5,1,4.0,WHOPPER® Meal


In [9]:
df = full_data.copy()
df.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour,meal_of_day,order_hour_sin,order_hour_cos
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,1900-01-01 12:03:29,85,japanese garlic karaage don,6.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,1900-01-01 12:03:29,85,chicken cutlet don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,1900-01-01 12:03:29,85,beef sukiyaki don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,1900-01-01 12:03:29,85,japanese beef yakiniku don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,1900-01-01 12:03:29,85,teriyaki salmon don,8.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0


In [10]:
# add products rating to full_data
# One-line solution:
# Insert 'order_frequency' before 'product_rating'
full_data.insert(full_data.columns.get_loc('chain_id'), 'order_frequency', full_data100k['order_frequency'])

# Then insert 'product_rating' right after it
full_data.insert(full_data.columns.get_loc('chain_id'), 'product_rating', full_data100k['product_rating'])

print("✅ Done! order_frequency and product_rating added between unit_price and chain_id")
print(f"full_data now has {full_data.shape[1]} columns")

✅ Done! order_frequency and product_rating added between unit_price and chain_id
full_data now has 27 columns


## Inspect both dataframes

In [11]:
# Quick data exploration
print("Data shape:", full_data.shape)
print("\nUnique counts:")
print(f"Customers: {full_data['customer_id'].nunique()}")
print(f"Vendors: {full_data['vendor_id'].nunique()}")
print(f"Cuisines: {full_data['cuisine_origin'].nunique()}")
print(f"Orders: {full_data['order_id'].nunique()}")

print("\nRating distribution:")
print(full_data['product_rating'].value_counts().sort_index())

# Check vendor_ratings dataframe
print(f"\nVendor ratings shape: {full_data2.shape}")
print(f"Vendor ratings columns: {full_data2.columns.tolist()}")
print(f"Vendor ratings stats:")
print(full_data2['vendor_rating'].describe())

Data shape: (100000, 27)

Unique counts:
Customers: 11174
Vendors: 5818
Cuisines: 18
Orders: 49029

Rating distribution:
product_rating
1.0    10733
2.0    13319
3.0    17395
4.0    18578
5.0    39974
Name: count, dtype: int64

Vendor ratings shape: (49029, 14)
Vendor ratings columns: ['customer_id', 'customer_geohash', 'order_id', 'vendor_id', 'day_of_week', 'order_time', 'order_day', 'chain_id', 'vendor_geohash', 'cuisine_origin', 'vendor_rating', 'num_products', 'total_order_value', 'products_ordered']
Vendor ratings stats:
count    49029.000000
mean         4.073067
std          0.368664
min          2.500000
25%          3.900000
50%          4.000000
75%          4.400000
max          5.000000
Name: vendor_rating, dtype: float64


# 1. Implicit Feedback Matrix Features:
Weighted scores combining order frequency and ratings (Use orders as strong preference signals).

User-Vendor matrix: Interaction strength between customers and vendors (Customer’s favorite restaurants based on order frequency)

User-Cuisine matrix: Preference strength for cuisine types (“Who likes which types of food?")

In [12]:
#df.dropna(inplace=True)
df.isnull().sum()

customer_id          0
customer_geohash     0
order_id             0
vendor_id            0
product_id           0
day_of_week          0
order_time           0
order_day            0
name                32
unit_price           0
chain_id             0
vendor_geohash       0
cuisine_origin       0
is_weekend           0
dow_0                0
dow_1                0
dow_2                0
dow_3                0
dow_4                0
dow_5                0
dow_6                0
order_hour           0
meal_of_day          0
order_hour_sin       0
order_hour_cos       0
dtype: int64

In [13]:
df.dropna(inplace=True)
df.isnull().sum()

customer_id         0
customer_geohash    0
order_id            0
vendor_id           0
product_id          0
day_of_week         0
order_time          0
order_day           0
name                0
unit_price          0
chain_id            0
vendor_geohash      0
cuisine_origin      0
is_weekend          0
dow_0               0
dow_1               0
dow_2               0
dow_3               0
dow_4               0
dow_5               0
dow_6               0
order_hour          0
meal_of_day         0
order_hour_sin      0
order_hour_cos      0
dtype: int64

In [14]:
full_data.columns

Index(['customer_id', 'customer_geohash', 'order_id', 'vendor_id',
       'product_id', 'day_of_week', 'order_time', 'order_day', 'name',
       'unit_price', 'order_frequency', 'product_rating', 'chain_id',
       'vendor_geohash', 'cuisine_origin', 'is_weekend', 'dow_0', 'dow_1',
       'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6', 'order_hour',
       'meal_of_day', 'order_hour_sin', 'order_hour_cos'],
      dtype='object')

In [15]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 27 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   customer_id       100000 non-null  object 
 1   customer_geohash  100000 non-null  object 
 2   order_id          100000 non-null  int64  
 3   vendor_id         100000 non-null  object 
 4   product_id        100000 non-null  object 
 5   day_of_week       100000 non-null  int64  
 6   order_time        100000 non-null  object 
 7   order_day         100000 non-null  int64  
 8   name              99968 non-null   object 
 9   unit_price        100000 non-null  float64
 10  order_frequency   99999 non-null   float64
 11  product_rating    99999 non-null   float64
 12  chain_id          100000 non-null  object 
 13  vendor_geohash    100000 non-null  object 
 14  cuisine_origin    100000 non-null  object 
 15  is_weekend        100000 non-null  int64  
 16  dow_0             100

In [16]:
full_data2.head()

,customer_id,customer_geohash,order_id,vendor_id,day_of_week,order_time,order_day,chain_id,vendor_geohash,cuisine_origin,vendor_rating,num_products,total_order_value,products_ordered
0,00119c8178,w21zu,39095,e7cb5902,2,16:30:07,10 days,ef3142e8,w21zu,malaysian,3.5,1,5.6,Seafood Fried Rice
1,00198e01e4,w21z6,35939,02acaff6,0,13:57:35,85 days,8c51e46b,w21z6,chinese,3.9,4,8.0,"Bean Curd Skin 凉拌腐竹, Dried Beancurd 凉拌素鸡, Blac..."
2,001a5689fc,w21z3,48288,a2c60e71,0,20:10:35,78 days,dd69fe77,w21z3,singaporean,4.5,1,2.4,Carrot Cake (Melur)
3,001a5689fc,w21z3,48289,a3bc472c,4,20:50:40,40 days,c59edb7d,w21z9,snacks,4.0,1,1.6,Served without Fries
4,001a5689fc,w21z3,48290,de18b671,3,20:05:42,53 days,f88ffd2b,w21z3,american,4.5,1,4.0,WHOPPER® Meal


# Cell 1: Enhanced Data Preprocessing & Feature Engineering
### Purpose: This cell performs comprehensive data preprocessing, filtering, and feature engineering to create clean, structured data for the recommendation system. It merges datasets, filters low-quality interactions, and creates aggregated features for users and vendors.
What it does:

- Merges order data with vendor ratings and details

- Filters out low-rated orders (rating < 3) and inactive users/vendors

- Creates order-level aggregated features (total value, item count, etc.)

- Builds vendor profiles with cuisine, ratings, and activity metrics

- Ensures data quality by removing sparse entities

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from implicit.als import AlternatingLeastSquares
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

print("🚀 ENHANCED FOOD DELIVERY RECOMMENDER")
print("="*50)

# =============================================
# ENHANCED DATA PREPROCESSING
# =============================================

print("Step 1: Enhanced Data Preprocessing...")

def enhanced_data_preprocessing(full_data, full_data2, min_rating=3.5, min_user_orders=2, min_vendor_orders=10):
    """
    Enhanced preprocessing with activity filtering and rating optimization
    Now using min_rating=3.5 to focus on higher quality interactions
    """

    # Merge vendor ratings
    print("📊 Merging vendor ratings and order details...")
    full_data_enhanced = full_data.merge(
        full_data2[['order_id', 'vendor_id', 'vendor_rating', 'total_order_value', 'num_products']],
        on=['order_id', 'vendor_id'],
        how='left',
        suffixes=('', '_vendor')
    )
    print(f"   Enhanced data shape: {full_data_enhanced.shape}")

    # Filter by rating (OPTIMIZED: using 3.5+ stars for better quality)
    print(f"⭐ Filtering by rating >= {min_rating} (quality focus)...")
    data_filtered = full_data_enhanced[full_data_enhanced['product_rating'] >= min_rating].copy()
    print(f"   After rating filter: {data_filtered.shape}")

    # Create order-level features
    print("🔧 Creating order-level features...")
    order_level_data = data_filtered.groupby('order_id').agg({
        'customer_id': 'first',
        'vendor_id': 'first',
        'cuisine_origin': 'first',
        'day_of_week': 'first',
        'order_day': 'first',
        'order_hour': 'first',
        'meal_of_day': 'first',
        'is_weekend': 'first',
        'unit_price': 'sum',
        'product_id': 'count',
        'product_rating': 'mean',
        'vendor_rating': 'first',
        'order_frequency': 'first',
        'total_order_value': 'first',
        'num_products': 'first',
        'customer_geohash': 'first',
        'vendor_geohash': 'first'
    }).rename(columns={
        'product_id': 'num_items',
        'product_rating': 'avg_product_rating',
        'unit_price': 'total_order_value_calculated'
    }).reset_index()

    # Handle order value calculation
    order_level_data['final_order_value'] = order_level_data['total_order_value'].fillna(
        order_level_data['total_order_value_calculated']
    )
    print(f"   Order-level data shape: {order_level_data.shape}")

    # FILTER LOW-ACTIVITY USERS AND VENDORS
    print(f"👥 Activity filtering: users >= {min_user_orders} orders, vendors >= {min_vendor_orders} orders...")

    user_orders = order_level_data['customer_id'].value_counts()
    active_users = user_orders[user_orders >= min_user_orders].index
    vendor_orders = order_level_data['vendor_id'].value_counts()
    active_vendors = vendor_orders[vendor_orders >= min_vendor_orders].index

    order_level_data = order_level_data[
        order_level_data['customer_id'].isin(active_users) &
        order_level_data['vendor_id'].isin(active_vendors)
    ]

    print(f"   After activity filtering: {order_level_data.shape}")
    print(f"   Active users: {len(active_users):,}, Active vendors: {len(active_vendors):,}")

    # Create enhanced vendor features
    print("🏪 Creating vendor features with rating focus...")
    vendor_features = data_filtered.groupby('vendor_id').agg({
        'cuisine_origin': 'first',
        'vendor_geohash': 'first',
        'vendor_rating': 'mean',
        'product_rating': ['mean', 'count'],
        'order_id': 'nunique'
    }).reset_index()

    vendor_features.columns = [
        'vendor_id', 'cuisine_origin', 'vendor_geohash',
        'avg_vendor_rating', 'avg_product_rating', 'total_products', 'total_orders'
    ]

    # Filter vendors to only include those with good ratings (3.5+)
    vendor_features = vendor_features[vendor_features['avg_vendor_rating'] >= 3.5]
    print(f"   Vendor features shape: {vendor_features.shape}")

    return order_level_data, vendor_features

# Apply enhanced preprocessing with 3.5+ star rating filter
order_level_data, vendor_features = enhanced_data_preprocessing(
    full_data, full_data2, min_rating=3.5, min_user_orders=2, min_vendor_orders=10
)

print("✅ Data preprocessing completed successfully!")
print(f"📈 Final dataset: {order_level_data.shape[0]:,} orders, {order_level_data['customer_id'].nunique():,} users, {order_level_data['vendor_id'].nunique():,} vendors")

🚀 ENHANCED FOOD DELIVERY RECOMMENDER
Step 1: Enhanced Data Preprocessing...
📊 Merging vendor ratings and order details...
   Enhanced data shape: (100000, 30)
⭐ Filtering by rating >= 3.5 (quality focus)...
   After rating filter: (58552, 30)
🔧 Creating order-level features...
   Order-level data shape: (35622, 19)
👥 Activity filtering: users >= 2 orders, vendors >= 10 orders...
   After activity filtering: (17655, 19)
   Active users: 5,156, Active vendors: 1,328
🏪 Creating vendor features with rating focus...
   Vendor features shape: (5394, 7)
✅ Data preprocessing completed successfully!
📈 Final dataset: 17,655 orders, 4,283 users, 1,328 vendors


# Cell 2: Enhanced Interaction Matrices & Preference Scoring
### Purpose: This cell creates sophisticated interaction matrices that capture user preferences using multiple factors including frequency, ratings, recency, and monetary value. These matrices form the foundation for both collaborative and content-based filtering.

In [ ]:
# =============================================
# ENHANCED INTERACTION MATRICES
# =============================================

print("\nStep 2: Building Enhanced Interaction Matrices...")

def create_enhanced_interaction_matrices(df):
    """
    Create interaction matrices with optimized scoring weights
    Enhanced with better factor balancing and temporal patterns
    """

    print("   Creating user-vendor interactions with multi-factor scoring...")

    # 1. Enhanced User-Vendor Matrix with optimized factor weights
    user_vendor_interactions = df.groupby(['customer_id', 'vendor_id']).agg({
        'num_items': 'sum',
        'avg_product_rating': 'mean',
        'vendor_rating': 'mean',
        'order_id': 'count',
        'final_order_value': 'mean',  # Monetary value
        'order_day': 'max'  # Recency
    }).reset_index()

    # Calculate recency score (more recent = higher score)
    max_order_day = user_vendor_interactions['order_day'].max()
    user_vendor_interactions['recency_score'] = 1 - (
        (max_order_day - user_vendor_interactions['order_day']) /
        (max_order_day - user_vendor_interactions['order_day'].min() + 1)
    )

    # OPTIMIZED: Enhanced weighted score with balanced factors
    # Weights: Frequency(40%), Product Quality(20%), Vendor Quality(20%), Recency(10%), Monetary(10%)
    user_vendor_interactions['enhanced_score'] = (
        (user_vendor_interactions['order_id'] / user_vendor_interactions['order_id'].max()) * 0.4 +  # Frequency
        (user_vendor_interactions['avg_product_rating'] / 5.0) * 0.2 +  # Product quality
        (user_vendor_interactions['vendor_rating'] / 5.0) * 0.2 +  # Vendor quality
        user_vendor_interactions['recency_score'] * 0.1 +  # Recency
        (user_vendor_interactions['final_order_value'] / user_vendor_interactions['final_order_value'].max()) * 0.1  # Monetary
    )

    # Ensure scores are normalized between 0-1
    user_vendor_interactions['enhanced_score'] = user_vendor_interactions['enhanced_score'].clip(0, 1)

    user_vendor_matrix = user_vendor_interactions.pivot_table(
        index='customer_id',
        columns='vendor_id',
        values='enhanced_score',
        fill_value=0
    )

    print("   Creating user-cuisine preference matrix...")

    # 2. User-Cuisine Matrix with temporal patterns and meal preferences
    user_cuisine_interactions = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'order_id': 'count',
        'avg_product_rating': 'mean',
        'vendor_rating': 'mean',
        'meal_of_day': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'lunch',
        'is_weekend': 'mean'  # Weekend preference for this cuisine
    }).reset_index()

    # Enhanced cuisine preference score
    user_cuisine_interactions['preference_score'] = (
        (user_cuisine_interactions['order_id'] / user_cuisine_interactions.groupby('customer_id')['order_id'].transform('sum')) * 0.5 +  # Relative frequency
        (user_cuisine_interactions['avg_product_rating'] / 5.0) * 0.3 +  # Quality
        (user_cuisine_interactions['vendor_rating'] / 5.0) * 0.2  # Vendor quality
    )

    user_cuisine_matrix = user_cuisine_interactions.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='preference_score',
        fill_value=0
    )

    return {
        'user_vendor': user_vendor_matrix,
        'user_cuisine': user_cuisine_matrix,
        'raw_interactions': user_vendor_interactions,
        'cuisine_interactions': user_cuisine_interactions
    }

# Create enhanced matrices
matrices = create_enhanced_interaction_matrices(order_level_data)
user_vendor_matrix = matrices['user_vendor']
user_cuisine_matrix = matrices['user_cuisine']
user_vendor_interactions = matrices['raw_interactions']

print(f"✅ User-Vendor Matrix: {user_vendor_matrix.shape}")
print(f"✅ User-Cuisine Matrix: {user_cuisine_matrix.shape}")

# Calculate and display matrix statistics
total_interactions = user_vendor_matrix.shape[0] * user_vendor_matrix.shape[1]
actual_interactions = (user_vendor_matrix > 0).sum().sum()
sparsity = 1 - (actual_interactions / total_interactions)

print(f"📊 Matrix Statistics:")
print(f"   - Total possible interactions: {total_interactions:,}")
print(f"   - Actual interactions: {actual_interactions:,}")
print(f"   - Data Sparsity: {sparsity:.4f} ({sparsity*100:.2f}%)")
print(f"   - Average user interactions: {actual_interactions / user_vendor_matrix.shape[0]:.2f}")
print(f"   - Score range: {user_vendor_interactions['enhanced_score'].min():.3f} - {user_vendor_interactions['enhanced_score'].max():.3f}")

# Show sample of interaction scores
print(f"   - Sample enhanced scores:")
sample_scores = user_vendor_interactions['enhanced_score'].describe()
print(f"     Min: {sample_scores['min']:.3f}, Mean: {sample_scores['mean']:.3f}, Max: {sample_scores['max']:.3f}")


Step 2: Building Enhanced Interaction Matrices...
   Creating user-vendor interactions with multi-factor scoring...
   Creating user-cuisine preference matrix...
✅ User-Vendor Matrix: (4283, 1328)
✅ User-Cuisine Matrix: (4283, 14)
📊 Matrix Statistics:
   - Total possible interactions: 5,687,824
   - Actual interactions: 11,014
   - Data Sparsity: 0.9981 (99.81%)
   - Average user interactions: 2.57
   - Score range: 0.286 - 0.885
   - Sample enhanced scores:
     Min: 0.286, Mean: 0.416, Max: 0.885


# Cell 3: Enhanced Collaborative Filtering with ALS
### Purpose: This cell trains the Alternating Least Squares (ALS) model for collaborative filtering, which learns latent factors from user-vendor interactions to predict user preferences. The enhanced version includes adaptive parameter tuning based on data sparsity and better model configuration.

In [ ]:
# =============================================
# ENHANCED COLLABORATIVE FILTERING
# =============================================

print("\nStep 3: Training Enhanced Collaborative Filtering...")

def train_enhanced_als(interaction_matrix, factors=64, iterations=20, regularization=0.1):
    """Train ALS with better parameters for sparse data"""

    sparse_matrix = csr_matrix(interaction_matrix.values)

    # Adjust parameters based on sparsity
    n_users, n_items = interaction_matrix.shape
    sparsity = 1 - (sparse_matrix.nnz / (n_users * n_items))

    # Use more factors and regularization for sparse data
    if sparsity > 0.99:
        factors = 32  # Reduce factors for very sparse data
        regularization = 0.2

    print(f"Training ALS: {factors} factors, {iterations} iterations, regularization={regularization}")

    als_model = AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=regularization,
        random_state=42,
        use_gpu=False
    )

    als_model.fit(sparse_matrix * 10)  # Scale to improve performance

    return als_model, sparse_matrix

als_model, sparse_matrix = train_enhanced_als(user_vendor_matrix)
print(f"ALS Model: {als_model.user_factors.shape} user factors, {als_model.item_factors.shape} item factors")


Step 3: Training Enhanced Collaborative Filtering...
Training ALS: 32 factors, 20 iterations, regularization=0.2


  0%|          | 0/20 [00:00<?, ?it/s]

ALS Model: (4283, 32) user factors, (1328, 32) item factors


# Cell 4: Enhanced User Profiling & Behavioral Clustering
### Purpose: This cell creates comprehensive user profiles by analyzing cuisine preferences, behavioral patterns, and ordering habits. It then performs customer segmentation using clustering to group similar users for better personalized recommendations.

In [ ]:
# # =============================================
# # ENHANCED USER PROFILES & BEHAVIORAL CLUSTERING
# # =============================================

# print("\nStep 4: Building Enhanced User Profiles & Behavioral Clustering...")

# def create_enhanced_user_profiles(df):
#     """
#     Create comprehensive user profiles with cuisine preferences and behavioral patterns
#     Enhanced with better preference scoring and temporal analysis
#     """

#     print("   Analyzing user cuisine preferences...")

#     # 1. Enhanced Cuisine Preferences with weighted scoring
#     cuisine_prefs = df.groupby(['customer_id', 'cuisine_origin']).agg({
#         'order_id': 'count',
#         'avg_product_rating': 'mean',
#         'vendor_rating': 'mean',
#         'final_order_value': 'mean',
#         'meal_of_day': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'lunch',
#         'order_day': 'max'  # For recency
#     }).reset_index()

#     # Calculate recency for cuisine preferences
#     max_day = cuisine_prefs['order_day'].max()
#     cuisine_prefs['cuisine_recency'] = 1 - ((max_day - cuisine_prefs['order_day']) / (max_day - cuisine_prefs['order_day'].min() + 1))

#     # Enhanced preference scoring with recency and value factors
#     cuisine_prefs['preference_score'] = (
#         (cuisine_prefs['order_id'] / cuisine_prefs.groupby('customer_id')['order_id'].transform('sum')) * 0.4 +  # Relative frequency
#         (cuisine_prefs['avg_product_rating'] / 5.0) * 0.3 +  # Quality preference
#         (cuisine_prefs['vendor_rating'] / 5.0) * 0.2 +  # Vendor quality
#         cuisine_prefs['cuisine_recency'] * 0.1  # Recency bias
#     )

#     # Normalize by user to get preference distribution
#     user_totals = cuisine_prefs.groupby('customer_id')['preference_score'].transform('sum')
#     cuisine_prefs['normalized_score'] = cuisine_prefs['preference_score'] / (user_totals + 1e-8)  # Avoid division by zero

#     user_cuisine_profiles = cuisine_prefs.pivot_table(
#         index='customer_id',
#         columns='cuisine_origin',
#         values='normalized_score',
#         fill_value=0
#     )

#     print("   Building behavioral profiles...")

#     # 2. Enhanced Behavioral Profiles with comprehensive features
#     behavioral_profiles = df.groupby('customer_id').agg({
#         'order_hour': ['mean', 'std', lambda x: x.mode()[0] if len(x.mode()) > 0 else 12],  # Peak hour
#         'is_weekend': 'mean',
#         'day_of_week': lambda x: x.mode()[0] if len(x.mode()) > 0 else 3,  # Favorite day
#         'meal_of_day': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'lunch',
#         'final_order_value': ['mean', 'std', 'max', 'min'],
#         'num_items': ['mean', 'sum'],
#         'order_id': 'count',
#         'order_frequency': 'mean',
#         'vendor_rating': 'mean',
#         'avg_product_rating': 'mean',
#         'order_day': ['min', 'max'],  # Customer tenure
#         'vendor_id': 'nunique'  # Variety seeking
#     }).reset_index()

#     # Flatten column names
#     behavioral_profiles.columns = [
#         'customer_id', 'avg_order_hour', 'std_order_hour', 'peak_order_hour',
#         'weekend_ratio', 'fav_day', 'fav_meal', 'avg_order_value', 'std_order_value',
#         'max_order_value', 'min_order_value', 'avg_items_per_order', 'total_items',
#         'total_orders', 'avg_order_frequency', 'avg_vendor_rating_choice',
#         'avg_product_rating_given', 'first_order_day', 'last_order_day', 'unique_vendors_tried'
#     ]

#     # Calculate enhanced behavioral metrics
#     behavioral_profiles['customer_tenure'] = behavioral_profiles['last_order_day'] - behavioral_profiles['first_order_day']
#     behavioral_profiles['order_value_consistency'] = 1 / (behavioral_profiles['std_order_value'] + 1)  # Higher = more consistent
#     behavioral_profiles['variety_seeking'] = behavioral_profiles['unique_vendors_tried'] / behavioral_profiles['total_orders']
#     behavioral_profiles['avg_order_size'] = behavioral_profiles['total_items'] / behavioral_profiles['total_orders']

#     print("   Combining profiles...")

#     # 3. Combine profiles
#     user_profiles = user_cuisine_profiles.reset_index()
#     user_profiles = user_profiles.merge(behavioral_profiles, on='customer_id', how='left')

#     return user_profiles.fillna(0)

# # Create enhanced user profiles
# user_profiles = create_enhanced_user_profiles(order_level_data)
# print(f"✅ Enhanced user profiles shape: {user_profiles.shape}")

# # Enhanced Clustering with Better Feature Selection
# def create_enhanced_clusters(user_profiles, max_clusters=8):
#     """
#     Create customer segments using key behavioral features
#     Enhanced with outlier detection and optimal cluster selection
#     """

#     print("   Performing customer segmentation...")

#     # Select key behavioral features for clustering
#     key_features = [
#         'avg_order_value', 'total_orders', 'weekend_ratio', 'avg_order_hour',
#         'avg_items_per_order', 'customer_tenure', 'avg_vendor_rating_choice',
#         'variety_seeking', 'order_value_consistency'
#     ]

#     # Only use features that exist and have variance
#     available_features = []
#     for f in key_features:
#         if f in user_profiles.columns:
#             if user_profiles[f].std() > 0.001:  # Filter near-constant features
#                 available_features.append(f)

#     print(f"   Using {len(available_features)} features for clustering: {available_features}")

#     if len(available_features) < 3:
#         print("⚠️  Not enough features for meaningful clustering")
#         user_profiles['user_cluster'] = 0
#         return user_profiles, None, None

#     cluster_data = user_profiles[available_features].fillna(0)

#     # Remove outliers using IQR method (keep 95% of data)
#     if len(cluster_data) > 10:
#         Q1 = cluster_data.quantile(0.05)  # Use 5th percentile instead of 25th to be less aggressive
#         Q3 = cluster_data.quantile(0.95)  # Use 95th percentile
#         IQR = Q3 - Q1
#         outlier_mask = ~((cluster_data < (Q1 - 1.5 * IQR)) | (cluster_data > (Q3 + 1.5 * IQR))).any(axis=1)
#         cluster_data_clean = cluster_data[outlier_mask]
#         print(f"   Removed {len(cluster_data) - len(cluster_data_clean)} outliers")
#     else:
#         cluster_data_clean = cluster_data

#     if len(cluster_data_clean) < 10:
#         print("⚠️  Not enough data for clustering after outlier removal")
#         user_profiles['user_cluster'] = 0
#         return user_profiles, None, None

#     # Standardize features
#     scaler = StandardScaler()
#     cluster_data_scaled = scaler.fit_transform(cluster_data_clean)

#     # Find optimal number of clusters using silhouette score
#     print("   Finding optimal cluster count...")
#     best_score = -1
#     best_n = 2
#     silhouette_scores = []

#     max_possible = min(max_clusters, len(cluster_data_scaled) // 5)
#     cluster_range = range(2, max_possible + 1)

#     for n in cluster_range:
#         if n >= len(cluster_data_scaled):
#             continue

#         kmeans = KMeans(n_clusters=n, random_state=42, n_init=15)
#         labels = kmeans.fit_predict(cluster_data_scaled)

#         if len(np.unique(labels)) > 1:  # Need at least 2 clusters for silhouette score
#             score = silhouette_score(cluster_data_scaled, labels)
#             silhouette_scores.append(score)

#             if score > best_score:
#                 best_score = score
#                 best_n = n

#     print(f"   Optimal clusters: {best_n} (silhouette: {best_score:.3f})")

#     # Apply final clustering
#     kmeans = KMeans(n_clusters=best_n, random_state=42, n_init=15)
#     user_profiles['user_cluster'] = -1  # Default for outliers/non-clustered

#     # Assign clusters to clean data points
#     valid_indices = cluster_data_clean.index
#     cluster_labels = kmeans.fit_predict(cluster_data_scaled)
#     user_profiles.loc[valid_indices, 'user_cluster'] = cluster_labels

#     # Add cluster sizes information
#     cluster_sizes = user_profiles[user_profiles['user_cluster'] != -1]['user_cluster'].value_counts().sort_index()
#     print(f"   Cluster sizes: {dict(cluster_sizes)}")

#     return user_profiles, kmeans, scaler

# # Perform enhanced clustering
# user_profiles_with_clusters, cluster_model, cluster_scaler = create_enhanced_clusters(user_profiles)

# print(f"✅ Clustering completed!")
# print(f"📊 User segmentation summary:")
# cluster_summary = user_profiles_with_clusters['user_cluster'].value_counts().sort_index()
# for cluster_id, count in cluster_summary.items():
#     percentage = (count / len(user_profiles_with_clusters)) * 100
#     print(f"   Cluster {cluster_id}: {count} users ({percentage:.1f}%)")

# NEW Cell 4 simplified (no clustering)

In [ ]:
# =============================================
# SIMPLIFIED USER PROFILES
# =============================================

print("\nStep 4: Creating Simplified User Profiles...")

def create_simplified_user_profiles(df):
    """Create essential user profiles without clustering complexity"""

    print("   Building user cuisine preferences...")

    # 1. Cuisine Preferences Only (most important for content-based)
    cuisine_prefs = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'order_id': 'count',
        'avg_product_rating': 'mean'
    }).reset_index()

    # Simple preference scoring
    user_total_orders = cuisine_prefs.groupby('customer_id')['order_id'].transform('sum')
    cuisine_prefs['preference_score'] = cuisine_prefs['order_id'] / user_total_orders

    user_cuisine_profiles = cuisine_prefs.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='preference_score',
        fill_value=0
    )

    print("   Building basic behavioral profiles...")

    # 2. Basic Behavioral Features - using correct column names
    behavioral_profiles = df.groupby('customer_id').agg({
        'order_id': 'count',  # Total orders
        'final_order_value': 'mean',  # Average order value
        'vendor_id': 'nunique',  # Unique vendors tried
        'avg_product_rating': 'mean'  # Average rating given
    }).reset_index()

    # Rename columns for clarity
    behavioral_profiles.columns = [
        'customer_id',
        'total_orders',
        'avg_order_value',
        'unique_vendors',
        'avg_rating_given'
    ]

    # 3. Combine profiles
    user_profiles = user_cuisine_profiles.reset_index()
    user_profiles = user_profiles.merge(behavioral_profiles, on='customer_id', how='left')

    print(f"✅ Simplified user profiles: {user_profiles.shape}")
    return user_profiles.fillna(0)

user_profiles = create_simplified_user_profiles(order_level_data)


Step 4: Creating Simplified User Profiles...
   Building user cuisine preferences...
   Building basic behavioral profiles...
✅ Simplified user profiles: (4283, 19)


# Cell 5: Enhanced Hybrid Recommender with Optimized Ranking
### Purpose: This cell creates the core hybrid recommendation engine that combines collaborative filtering, content-based filtering, and popularity signals with optimized weights and ranking. The enhanced version includes better score normalization, improved vendor rating filtering, and optimized ranking for evaluation metrics.

In [ ]:
# # =============================================
# # ENHANCED HYBRID RECOMMENDER (OPTIMIZED RANKING)
# # =============================================

# print("\nStep 5: Enhanced Hybrid Recommender with Optimized Ranking...")

# class EnhancedHybridRecommender:
#     def __init__(self, als_model, user_profiles, vendor_features, interaction_matrices):
#         self.als_model = als_model
#         self.user_profiles = user_profiles
#         self.vendor_features = vendor_features.set_index('vendor_id')
#         self.matrices = interaction_matrices
#         self.user_vendor_matrix = interaction_matrices['user_vendor']

#         # Filter vendors to only include those with 3.5+ stars (OPTIMIZED)
#         self.vendor_features = self.vendor_features[self.vendor_features['avg_vendor_rating'] >= 3.5]

#         # Align vendor features with interaction matrix
#         self.vendor_features = self._align_vendor_features()

#         # Enhanced fallback scores with rating focus
#         self.fallback_scores = self._compute_enhanced_fallback_scores()
#         self.user_activity_levels = self._compute_user_activity_levels()

#         print(f"✅ Recommender initialized with {len(self.vendor_features)} quality vendors (3.5+ stars)")

#     def _align_vendor_features(self):
#         """Ensure vendor features align with interaction matrix columns"""
#         matrix_vendors = set(self.user_vendor_matrix.columns)
#         feature_vendors = set(self.vendor_features.index)

#         # Keep only vendors that are in both sets
#         common_vendors = matrix_vendors.intersection(feature_vendors)

#         # Filter vendor features to common vendors
#         aligned_features = self.vendor_features.loc[list(common_vendors)]

#         print(f"   Aligned vendor features: {len(aligned_features)} vendors")
#         return aligned_features

#     def _compute_enhanced_fallback_scores(self):
#         """Compute fallback scores with proper alignment and rating focus"""
#         # Get popularity scores for aligned vendors only
#         popularity = self.user_vendor_matrix.sum(axis=0)
#         popularity = popularity[popularity.index.isin(self.vendor_features.index)]

#         # Get vendor ratings for aligned vendors (OPTIMIZED: emphasize ratings)
#         vendor_ratings = self.vendor_features['avg_vendor_rating']

#         # Normalize scores
#         pop_norm = popularity / popularity.max() if popularity.max() > 0 else popularity
#         rating_norm = vendor_ratings / 5.0

#         # Include novelty (inverse of popularity)
#         novelty = 1 - pop_norm

#         # OPTIMIZED: Combined score with more weight on ratings (40% rating, 30% popularity, 20% novelty, 10% product quality)
#         product_quality = self.vendor_features['avg_product_rating'] / 5.0
#         fallback_scores = (
#             0.3 * pop_norm +
#             0.4 * rating_norm +
#             0.2 * novelty +
#             0.1 * product_quality
#         )

#         return pd.DataFrame({
#             'vendor_id': fallback_scores.index,
#             'fallback_score': fallback_scores.values
#         }).sort_values('fallback_score', ascending=False)

#     def _compute_user_activity_levels(self):
#         """Compute user activity levels for dynamic weighting"""
#         user_orders = self.user_vendor_matrix.sum(axis=1)
#         if len(user_orders) == 0:
#             return {'low': 1, 'medium': 2}

#         return {
#             'low': user_orders.quantile(0.33),
#             'medium': user_orders.quantile(0.66)
#         }

#     def _get_dynamic_weights(self, user_id):
#         """Get dynamic weights based on user activity - OPTIMIZED for better ranking"""
#         if user_id not in self.user_vendor_matrix.index:
#             # New users: more weight to content and popularity
#             return 0.1, 0.5, 0.4

#         user_activity = self.user_vendor_matrix.loc[user_id].sum()

#         if user_activity <= self.user_activity_levels['low']:
#             return 0.3, 0.4, 0.3  # Low activity: balanced
#         elif user_activity <= self.user_activity_levels['medium']:
#             return 0.6, 0.3, 0.1  # Medium activity: more CF (OPTIMIZED)
#         else:
#             return 0.7, 0.2, 0.1  # High activity: mostly CF

#     def generate_enhanced_recommendations(self, user_id, top_n=10, novelty_boost=0.2):
#         """Generate recommendations with optimized scoring and ranking"""
#         cf_weight, cb_weight, pop_weight = self._get_dynamic_weights(user_id)

#         print(f"   User {user_id}: CF={cf_weight:.1f}, CB={cb_weight:.1f}, POP={pop_weight:.1f}")

#         # Get CF scores
#         if user_id in self.user_vendor_matrix.index:
#             user_idx = self.user_vendor_matrix.index.get_loc(user_id)
#             cf_scores = self.als_model.item_factors.dot(self.als_model.user_factors[user_idx])

#             # Create candidates dataframe with proper alignment
#             candidates = pd.DataFrame({
#                 'vendor_id': self.user_vendor_matrix.columns,
#                 'cf_score': cf_scores
#             })
#         else:
#             # For new users, use popularity-based scores
#             cf_scores = self.user_vendor_matrix.sum(axis=0).values
#             candidates = pd.DataFrame({
#                 'vendor_id': self.user_vendor_matrix.columns,
#                 'cf_score': cf_scores
#             })

#         # Get top candidates for efficiency (wider net for better ranking)
#         candidate_pool_size = min(top_n * 20, len(candidates))
#         candidates = candidates.nlargest(candidate_pool_size, 'cf_score')

#         # Enhanced CB scores with temporal patterns
#         cb_scores = self._compute_enhanced_cb_scores(user_id, candidates['vendor_id'])
#         candidates['cb_score'] = candidates['vendor_id'].map(cb_scores)

#         # Popularity scores (only for vendors in candidates)
#         pop_scores = self.user_vendor_matrix.sum(axis=0)
#         candidates['pop_score'] = candidates['vendor_id'].map(
#             lambda x: pop_scores[x] / pop_scores.max() if pop_scores.max() > 0 else pop_scores[x]
#         ).fillna(0)

#         # Vendor quality (emphasize 4+ star vendors)
#         candidates['quality_score'] = candidates['vendor_id'].map(
#             lambda x: self.vendor_features.loc[x, 'avg_vendor_rating'] if x in self.vendor_features.index else 0
#         ).fillna(0)

#         # Boost vendors with 4+ stars
#         candidates['rating_boost'] = candidates['quality_score'].apply(
#             lambda x: 0.2 if x >= 4.0 else (0.1 if x >= 3.5 else 0)
#         )

#         # Novelty scores (penalize vendors user has ordered from)
#         user_ordered = set()
#         if user_id in self.matrices['raw_interactions']['customer_id'].values:
#             user_ordered = set(self.matrices['raw_interactions'][
#                 self.matrices['raw_interactions']['customer_id'] == user_id
#             ]['vendor_id'])

#         candidates['novelty_score'] = candidates['vendor_id'].apply(
#             lambda x: novelty_boost if x not in user_ordered else -0.3  # Penalize previously ordered
#         )

#         # OPTIMIZED: Better normalization using min-max scaling
#         for col in ['cf_score', 'cb_score', 'pop_score', 'quality_score']:
#             min_val = candidates[col].min()
#             max_val = candidates[col].max()
#             if max_val > min_val:
#                 candidates[f'{col}_norm'] = (candidates[col] - min_val) / (max_val - min_val)
#             else:
#                 candidates[f'{col}_norm'] = 0

#         # OPTIMIZED FINAL SCORING with better weight distribution
#         candidates['final_score'] = (
#             cf_weight * candidates['cf_score_norm'] +
#             cb_weight * candidates['cb_score_norm'] +
#             pop_weight * candidates['pop_score_norm'] +
#             0.15 * candidates['quality_score_norm'] +  # Quality emphasis
#             candidates['rating_boost'] +  # Bonus for high-rated vendors
#             candidates['novelty_score']   # Novelty adjustment
#         )

#         # Apply cuisine diversity with optimized penalty
#         final_recommendations = self._apply_diversity(candidates, top_n)

#         # Ensure proper sorting by final_score (CRITICAL FIX)
#         final_recommendations = final_recommendations.sort_values('final_score', ascending=False).head(top_n)

#         print(f"   Generated {len(final_recommendations)} recommendations, score range: {final_recommendations['final_score'].min():.3f}-{final_recommendations['final_score'].max():.3f}")

#         return final_recommendations

#     def _compute_enhanced_cb_scores(self, user_id, candidate_vendors):
#         """Compute enhanced content-based scores with cuisine preferences"""
#         scores = {}

#         if user_id not in self.user_profiles['customer_id'].values:
#             # Return moderate scores for new users
#             return {vid: 0.4 for vid in candidate_vendors}

#         user_profile = self.user_profiles[self.user_profiles['customer_id'] == user_id].iloc[0]

#         for vendor_id in candidate_vendors:
#             if vendor_id not in self.vendor_features.index:
#                 scores[vendor_id] = 0
#                 continue

#             vendor_cuisine = self.vendor_features.loc[vendor_id, 'cuisine_origin']
#             vendor_rating = self.vendor_features.loc[vendor_id, 'avg_vendor_rating']

#             # Base cuisine preference from user profile
#             cuisine_score = user_profile.get(vendor_cuisine, 0.1)  # Default to 0.1 for unknown cuisines

#             # Boost for high-rated vendors in preferred cuisines
#             rating_boost = 0.2 if vendor_rating >= 4.0 else (0.1 if vendor_rating >= 3.5 else 0)

#             # Temporal pattern matching (simplified)
#             temporal_boost = self._compute_temporal_boost(user_profile, vendor_id)

#             # Combined CB score
#             scores[vendor_id] = cuisine_score * (1 + rating_boost + temporal_boost)

#         return scores

#     def _compute_temporal_boost(self, user_profile, vendor_id):
#         """Compute temporal pattern boost based on user behavior"""
#         # Simplified implementation - can be enhanced with actual order patterns
#         return 0.05  # Small consistent boost

#     def _apply_diversity(self, candidates, top_n):
#         """Apply diversity to avoid same-cuisine overload with optimized penalty"""
#         final_recs = []
#         cuisine_counts = {}

#         # Get cuisine for each vendor
#         vendor_cuisines = {}
#         for vendor_id in candidates['vendor_id']:
#             if vendor_id in self.vendor_features.index:
#                 vendor_cuisines[vendor_id] = self.vendor_features.loc[vendor_id, 'cuisine_origin']
#             else:
#                 vendor_cuisines[vendor_id] = 'unknown'

#         # Sort by final_score first
#         sorted_candidates = candidates.nlargest(min(top_n * 5, len(candidates)), 'final_score')

#         for idx, row in sorted_candidates.iterrows():
#             vendor_id = row['vendor_id']
#             cuisine = vendor_cuisines.get(vendor_id, 'unknown')
#             count = cuisine_counts.get(cuisine, 0)

#             # OPTIMIZED: Apply softer diversity penalty
#             penalty = 1.0 / (1.0 + 0.15 * count)  # Reduced from 0.3 to 0.15
#             diversified_score = row['final_score'] * penalty

#             # Get vendor details
#             vendor_rating = self.vendor_features.loc[vendor_id, 'avg_vendor_rating'] if vendor_id in self.vendor_features.index else 0

#             final_recs.append({
#                 'vendor_id': vendor_id,
#                 'final_score': diversified_score,
#                 'cuisine': cuisine,
#                 'vendor_rating': vendor_rating,
#                 'original_score': row['final_score']
#             })

#             cuisine_counts[cuisine] = count + 1

#             if len(final_recs) >= top_n * 2:  # Limit for efficiency
#                 break

#         result_df = pd.DataFrame(final_recs)
#         if len(result_df) > 0:
#             # Final sort by diversified score
#             return result_df.nlargest(min(top_n, len(result_df)), 'final_score')
#         else:
#             return pd.DataFrame(columns=['vendor_id', 'final_score', 'cuisine', 'vendor_rating', 'original_score'])

# # Initialize enhanced recommender with optimized ranking
# enhanced_recommender = EnhancedHybridRecommender(als_model, user_profiles_with_clusters, vendor_features, matrices)
# print("✅ Enhanced hybrid recommender with optimized ranking ready!")

# NEW Cell 5: OPTIMIZED HYBRID RECOMMENDER WITH IMPROVEMENTS

In [ ]:
# =============================================
# OPTIMIZED HYBRID RECOMMENDER (IMPROVED RANKING)
# =============================================

print("\nStep 5: Creating Optimized Hybrid Recommender with Ranking Improvements...")

class OptimizedHybridRecommender:
    def __init__(self, als_model, user_profiles, vendor_features, interaction_matrices):
        self.als_model = als_model
        self.user_profiles = user_profiles
        self.vendor_features = vendor_features.set_index('vendor_id')
        self.matrices = interaction_matrices
        self.user_vendor_matrix = interaction_matrices['user_vendor']

        # Filter to quality vendors only (3.5+ stars)
        self.vendor_features = self.vendor_features[self.vendor_features['avg_vendor_rating'] >= 3.5]
        self.vendor_features = self._align_vendor_features()

        print(f"✅ Optimized recommender: {len(self.vendor_features)} quality vendors")

    def _align_vendor_features(self):
        """Ensure vendor features align with interaction matrix"""
        matrix_vendors = set(self.user_vendor_matrix.columns)
        feature_vendors = set(self.vendor_features.index)
        common_vendors = matrix_vendors.intersection(feature_vendors)
        return self.vendor_features.loc[list(common_vendors)]

    def _get_user_weights(self, user_id):
        """Simple weight assignment based on user activity"""
        if user_id not in self.user_vendor_matrix.index:
            return 0.1, 0.6, 0.3  # New user: focus on content & popularity

        user_activity = self.user_vendor_matrix.loc[user_id].sum()
        if user_activity < 3:  # Low activity
            return 0.3, 0.5, 0.2
        else:  # Active user
            return 0.6, 0.3, 0.1

    def generate_recommendations(self, user_id, top_n=10):
        """Generate optimized recommendations with proper ranking and diversity"""
        cf_weight, cb_weight, pop_weight = self._get_user_weights(user_id)

        # Get candidate vendors (wider pool for ranking)
        candidates = self._get_candidate_scores(user_id, top_n * 20)
        if len(candidates) == 0:
            return pd.DataFrame()

        # Apply content-based scoring
        candidates['cb_score'] = candidates['vendor_id'].map(
            self._compute_content_scores(user_id, candidates['vendor_id'])
        )

        # Apply popularity scoring
        pop_scores = self.user_vendor_matrix.sum(axis=0)
        candidates['pop_score'] = candidates['vendor_id'].map(
            lambda x: pop_scores.get(x, 0) / pop_scores.max() if pop_scores.max() > 0 else 0
        )

        # Normalize scores (0-1 range)
        for col in ['cf_score', 'cb_score', 'pop_score']:
            min_val, max_val = candidates[col].min(), candidates[col].max()
            if max_val > min_val:
                candidates[f'{col}_norm'] = (candidates[col] - min_val) / (max_val - min_val)
            else:
                candidates[f'{col}_norm'] = 0.5  # Neutral score

        # Add vendor details for diversity calculation
        candidates['cuisine'] = candidates['vendor_id'].map(
            lambda x: self.vendor_features.loc[x, 'cuisine_origin'] if x in self.vendor_features.index else 'unknown'
        )
        candidates['vendor_rating'] = candidates['vendor_id'].map(
            lambda x: self.vendor_features.loc[x, 'avg_vendor_rating'] if x in self.vendor_features.index else 0
        )

        # ENHANCED FINAL SCORING with rating boost and controlled novelty
        candidates['rating_boost'] = candidates['vendor_rating'].apply(
            lambda x: 0.15 if x >= 4.0 else (0.05 if x >= 3.5 else 0)
        )

        # Reduced novelty boost (from 0.2 to 0.05)
        candidates['novelty_boost'] = candidates['vendor_id'].apply(
            lambda x: self._get_novelty_boost(user_id, x)
        )

        candidates['final_score'] = (
            cf_weight * candidates['cf_score_norm'] +
            cb_weight * candidates['cb_score_norm'] +
            pop_weight * candidates['pop_score_norm'] +
            candidates['rating_boost'] +      # Boost for high-rated vendors
            candidates['novelty_boost']       # Reduced novelty boost
        )

        # Apply diversity-aware ranking
        final_recommendations = self._apply_diversity_ranking(candidates, top_n)

        print(f"   Generated {len(final_recommendations)} recs | "
              f"Rating boost: {candidates['rating_boost'].mean():.3f} | "
              f"Novelty boost: {candidates['novelty_boost'].mean():.3f}")

        return final_recommendations[['vendor_id', 'final_score', 'cuisine', 'vendor_rating']]

    def _get_candidate_scores(self, user_id, candidate_pool):
        """Get collaborative filtering scores for candidates"""
        if user_id in self.user_vendor_matrix.index:
            user_idx = self.user_vendor_matrix.index.get_loc(user_id)
            cf_scores = self.als_model.item_factors.dot(self.als_model.user_factors[user_idx])

            candidates = pd.DataFrame({
                'vendor_id': self.user_vendor_matrix.columns,
                'cf_score': cf_scores
            })
        else:
            # New user - use popularity
            cf_scores = self.user_vendor_matrix.sum(axis=0).values
            candidates = pd.DataFrame({
                'vendor_id': self.user_vendor_matrix.columns,
                'cf_score': cf_scores
            })

        return candidates.nlargest(candidate_pool, 'cf_score')

    def _compute_content_scores(self, user_id, candidate_vendors):
        """Compute content-based scores using cuisine preferences"""
        scores = {}

        if user_id not in self.user_profiles['customer_id'].values:
            return {vid: 0.3 for vid in candidate_vendors}  # Default for new users

        user_profile = self.user_profiles[self.user_profiles['customer_id'] == user_id].iloc[0]

        for vendor_id in candidate_vendors:
            if vendor_id not in self.vendor_features.index:
                scores[vendor_id] = 0
                continue

            vendor_cuisine = self.vendor_features.loc[vendor_id, 'cuisine_origin']
            cuisine_score = user_profile.get(vendor_cuisine, 0.1)  # Default to 0.1 for unknown cuisines
            scores[vendor_id] = cuisine_score

        return scores

    def _get_novelty_boost(self, user_id, vendor_id):
        """Calculate novelty boost for vendors user hasn't ordered before"""
        if user_id not in self.matrices['raw_interactions']['customer_id'].values:
            return 0.05  # Small boost for new users

        user_ordered = set(self.matrices['raw_interactions'][
            self.matrices['raw_interactions']['customer_id'] == user_id
        ]['vendor_id'])

        return 0.05 if vendor_id not in user_ordered else 0  # Reduced from 0.2 to 0.05

    def _apply_diversity_ranking(self, candidates, top_n):
        """Apply diversity-aware ranking to improve cuisine variety"""
        # Sort by final_score first
        sorted_candidates = candidates.sort_values('final_score', ascending=False)

        final_recs = []
        selected_cuisines = set()
        cuisine_penalties = {}

        # Take wider pool for diversity selection
        candidate_pool = sorted_candidates.head(top_n * 3)

        for idx, row in candidate_pool.iterrows():
            if len(final_recs) >= top_n:
                break

            vendor_id = row['vendor_id']
            cuisine = row['cuisine']
            base_score = row['final_score']

            # Apply cuisine diversity penalty
            cuisine_count = cuisine_penalties.get(cuisine, 0)
            diversity_penalty = cuisine_count * 0.1  # Penalty for repeated cuisines
            diversified_score = base_score * (1 - diversity_penalty)

            final_recs.append({
                'vendor_id': vendor_id,
                'final_score': diversified_score,
                'cuisine': cuisine,
                'vendor_rating': row['vendor_rating'],
                'original_score': base_score,
                'diversity_penalty': diversity_penalty
            })

            cuisine_penalties[cuisine] = cuisine_count + 1
            selected_cuisines.add(cuisine)

        result_df = pd.DataFrame(final_recs)

        # CRITICAL: Final sort by diversified score to ensure proper ranking
        result_df = result_df.sort_values('final_score', ascending=False).head(top_n)

        print(f"   Diversity: {len(selected_cuisines)} cuisines | "
              f"Avg penalty: {result_df['diversity_penalty'].mean():.3f}")

        return result_df

# Initialize improved recommender
optimized_recommender = OptimizedHybridRecommender(als_model, user_profiles, vendor_features, matrices)
print("✅ Improved hybrid recommender with ranking fixes ready!")


Step 5: Creating Optimized Hybrid Recommender with Ranking Improvements...
✅ Optimized recommender: 1326 quality vendors
✅ Improved hybrid recommender with ranking fixes ready!


# NEW Cell 6: ENHANCED COMPREHENSIVE EVALUATION

In [ ]:
# =============================================
# ENHANCED COMPREHENSIVE EVALUATION
# =============================================

print("\nStep 6: Running Enhanced Comprehensive Evaluation...")

def calculate_ndcg(relevance_scores, k=10):
    """Calculate Normalized Discounted Cumulative Gain"""
    relevance = relevance_scores[:k]

    # Calculate DCG
    dcg = 0
    for i, rel in enumerate(relevance):
        dcg += (2 ** rel - 1) / np.log2(i + 2)

    # Calculate Ideal DCG
    ideal_relevance = sorted(relevance, reverse=True)
    idcg = 0
    for i, rel in enumerate(ideal_relevance):
        idcg += (2 ** rel - 1) / np.log2(i + 2)

    return dcg / idcg if idcg > 0 else 0

def calculate_mrr(relevance_scores, k=10):
    """Calculate Mean Reciprocal Rank"""
    relevance = relevance_scores[:k]
    for i, rel in enumerate(relevance):
        if rel > 0:
            return 1.0 / (i + 1)
    return 0

def enhanced_comprehensive_evaluation(recommender, order_level_data, sample_users=100, top_k=10):
    """Run complete evaluation with detailed analysis"""

    df_sorted = order_level_data.sort_values('order_day')
    user_order_counts = df_sorted['customer_id'].value_counts()
    eligible_users = user_order_counts[user_order_counts >= 3].index

    n_evaluate = min(sample_users, len(eligible_users))
    test_users = np.random.choice(eligible_users, n_evaluate, replace=False)

    print(f"Evaluating {len(test_users)} users with enhanced metrics...")

    metrics = {
        'precision@5': [], 'precision@10': [],
        'recall@5': [], 'recall@10': [],
        'ndcg@5': [], 'ndcg@10': [],
        'mrr@5': [], 'mrr@10': [],
        'novelty': [], 'diversity': [],
        'avg_position': [], 'top1_hits': []
    }

    user_breakdowns = []

    for i, user_id in enumerate(test_users):
        if i % 20 == 0:
            print(f"Progress: {i}/{len(test_users)}")

        user_orders = df_sorted[df_sorted['customer_id'] == user_id]
        if len(user_orders) < 2:
            continue

        train_orders = user_orders.iloc[:-1]
        test_order = user_orders.iloc[-1]
        actual_vendor = test_order['vendor_id']
        actual_cuisine = test_order['cuisine_origin']

        try:
            recommendations = recommender.generate_recommendations(user_id, top_n=top_k)
            if len(recommendations) == 0:
                continue

            rec_vendors = recommendations['vendor_id'].tolist()
            rec_cuisines = recommendations['cuisine'].tolist()
            rec_scores = recommendations['final_score'].tolist()

            # Calculate relevance
            relevance = [1 if v == actual_vendor else 0 for v in rec_vendors]

            # Basic metrics
            metrics['precision@5'].append(np.mean(relevance[:5]))
            metrics['precision@10'].append(np.mean(relevance[:10]))
            metrics['recall@5'].append(1 if any(relevance[:5]) else 0)
            metrics['recall@10'].append(1 if any(relevance[:10]) else 0)

            # Ranking metrics
            metrics['ndcg@5'].append(calculate_ndcg(relevance, 5))
            metrics['ndcg@10'].append(calculate_ndcg(relevance, 10))
            metrics['mrr@5'].append(calculate_mrr(relevance, 5))
            metrics['mrr@10'].append(calculate_mrr(relevance, 10))

            # Enhanced metrics
            user_history = set(train_orders['vendor_id'])
            metrics['novelty'].append(sum(1 for v in rec_vendors[:10] if v not in user_history) / 10)
            metrics['diversity'].append(len(set(rec_cuisines[:10])) / 10)

            # Position analysis
            if any(relevance):
                position = relevance.index(1) + 1
                metrics['avg_position'].append(position)
                metrics['top1_hits'].append(1 if position == 1 else 0)
            else:
                metrics['avg_position'].append(top_k + 1)  # Penalty for not found
                metrics['top1_hits'].append(0)

            # Store user breakdown for analysis
            if i < 3:  # First 3 users for detailed analysis
                user_breakdowns.append({
                    'user_id': user_id,
                    'total_orders': len(user_orders),
                    'actual_position': relevance.index(1) + 1 if any(relevance) else 'Not found',
                    'rec_scores_range': f"{min(rec_scores):.3f}-{max(rec_scores):.3f}",
                    'unique_cuisines': len(set(rec_cuisines)),
                    'found_in_top3': any(relevance[:3])
                })

        except Exception as e:
            continue

    # Calculate results
    results = {}
    for k, v in metrics.items():
        if v:
            results[k] = np.mean(v)
        else:
            results[k] = 0

    # Print comprehensive results with improvements
    print("\n" + "="*70)
    print("ENHANCED COMPREHENSIVE EVALUATION RESULTS")
    print("="*70)

    target_ranges = {
        'precision@10': (0.10, 0.30),
        'recall@10': (0.20, 0.40),
        'ndcg@10': (0.60, 0.85),
        'mrr@10': (0.30, 0.60)
    }

    print("\n📊 CORE PERFORMANCE METRICS:")
    print("-" * 50)
    for metric in ['precision@10', 'recall@10', 'ndcg@10', 'mrr@10']:
        if metric in results:
            value = results[metric]
            if metric in target_ranges:
                low, high = target_ranges[metric]
                status = "✅ WITHIN TARGET" if low <= value <= high else "⚠️ BELOW TARGET"
                improvement_needed = f"(Need +{low-value:.3f})" if value < low else ""
                print(f"{metric:15}: {value:.4f} [Target: {low:.2f}-{high:.2f}] {status} {improvement_needed}")

    print("\n🎯 RANKING ANALYSIS:")
    print("-" * 50)
    if 'avg_position' in results and 'top1_hits' in results:
        print(f"Average position of relevant items: {results['avg_position']:.2f}")
        print(f"Top-1 accuracy: {results['top1_hits']:.3f} ({results['top1_hits']*100:.1f}%)")
        print(f"Top-3 accuracy: {sum(1 for p in metrics['avg_position'] if p <= 3)/len(metrics['avg_position']):.3f}")

    print("\n🌟 QUALITY METRICS:")
    print("-" * 50)
    print(f"{'novelty':15}: {results.get('novelty', 0):.4f} (target: 0.7-0.9)")
    print(f"{'diversity':15}: {results.get('diversity', 0):.4f} (target: 0.3-0.5)")

    # User breakdown
    if user_breakdowns:
        print("\n🔍 SAMPLE USER BREAKDOWNS:")
        print("-" * 50)
        for user in user_breakdowns:
            print(f"User {user['user_id'][:8]} | Orders: {user['total_orders']} | "
                  f"Position: {user['actual_position']} | "
                  f"Cuisines: {user['unique_cuisines']} | "
                  f"Top3: {'Yes' if user['found_in_top3'] else 'No'}")

    # Improvement summary
    print("\n💡 IMPROVEMENTS IMPLEMENTED:")
    print("-" * 50)
    print("✅ Reduced novelty boost from 0.2 to 0.05")
    print("✅ Added rating boost for 4+ star vendors")
    print("✅ Enhanced diversity with cuisine penalties")
    print("✅ Fixed final ranking sort (critical for NDCG)")
    print("✅ Added position analysis for ranking quality")

    return results

# Run enhanced evaluation
print("Running evaluation with ranking improvements...")
final_metrics = enhanced_comprehensive_evaluation(optimized_recommender, order_level_data, sample_users=100)


Step 6: Running Enhanced Comprehensive Evaluation...
Running evaluation with ranking improvements...
Evaluating 100 users with enhanced metrics...
Progress: 0/100
   Diversity: 4 cuisines | Avg penalty: 0.160
   Generated 10 recs | Rating boost: 0.111 | Novelty boost: 0.048
   Diversity: 1 cuisines | Avg penalty: 0.450
   Generated 10 recs | Rating boost: 0.115 | Novelty boost: 0.049
   Diversity: 4 cuisines | Avg penalty: 0.100
   Generated 10 recs | Rating boost: 0.113 | Novelty boost: 0.049
   Diversity: 2 cuisines | Avg penalty: 0.360
   Generated 10 recs | Rating boost: 0.118 | Novelty boost: 0.048
   Diversity: 3 cuisines | Avg penalty: 0.130
   Generated 10 recs | Rating boost: 0.113 | Novelty boost: 0.048
   Diversity: 4 cuisines | Avg penalty: 0.120
   Generated 10 recs | Rating boost: 0.113 | Novelty boost: 0.049
   Diversity: 5 cuisines | Avg penalty: 0.050
   Generated 10 recs | Rating boost: 0.113 | Novelty boost: 0.049
   Diversity: 2 cuisines | Avg penalty: 0.290
   Gen

# Cell 6: Enhanced Evaluation with Comprehensive Metrics & Analysis
### Purpose: This cell performs comprehensive evaluation of the recommendation system using multiple metrics including Precision, Recall, NDCG, MRR, Novelty, and Diversity. The enhanced version includes detailed user analysis, vendor rating validation, and proper temporal splitting.

In [ ]:
# # =============================================
# # ENHANCED EVALUATION WITH COMPREHENSIVE METRICS
# # =============================================

# print("\nStep 6: Running Enhanced Evaluation with Comprehensive Metrics...")

# def calculate_ndcg(relevance_scores, k=10):
#     """Calculate Normalized Discounted Cumulative Gain"""
#     relevance = relevance_scores[:k]

#     # Calculate DCG
#     dcg = 0
#     for i, rel in enumerate(relevance):
#         dcg += (2 ** rel - 1) / np.log2(i + 2)  # i+2 because index starts at 0, and log base 2 of (i+2)

#     # Calculate Ideal DCG
#     ideal_relevance = sorted(relevance, reverse=True)
#     idcg = 0
#     for i, rel in enumerate(ideal_relevance):
#         idcg += (2 ** rel - 1) / np.log2(i + 2)

#     return dcg / idcg if idcg > 0 else 0

# def calculate_mrr(relevance_scores, k=10):
#     """Calculate Mean Reciprocal Rank"""
#     relevance = relevance_scores[:k]
#     for i, rel in enumerate(relevance):
#         if rel > 0:  # Found first relevant item
#             return 1.0 / (i + 1)
#     return 0

# def enhanced_evaluation(recommender, order_level_data, sample_users=100, top_k=10):
#     """
#     Enhanced evaluation with comprehensive metrics and user analysis
#     Includes Precision, Recall, NDCG, MRR, Novelty, Diversity, and Coverage
#     """

#     # Use temporal split but ensure minimum test interactions
#     df_sorted = order_level_data.sort_values('order_day')

#     # Only evaluate users with sufficient history
#     user_order_counts = df_sorted['customer_id'].value_counts()
#     eligible_users = user_order_counts[user_order_counts >= 3].index
#     test_users = np.random.choice(eligible_users, min(sample_users, len(eligible_users)), replace=False)

#     print(f"Evaluating {len(test_users)} users with sufficient history...")

#     # Enhanced metrics dictionary
#     metrics = {
#         'precision@5': [], 'precision@10': [],
#         'recall@5': [], 'recall@10': [],
#         'ndcg@5': [], 'ndcg@10': [],
#         'mrr@5': [], 'mrr@10': [],
#         'novelty': [], 'diversity': []
#     }

#     all_recommendations = []
#     user_analysis_data = []

#     for i, user_id in enumerate(test_users):
#         if i % 20 == 0:
#             print(f"Progress: {i}/{len(test_users)}")

#         # Get user's last order as test
#         user_orders = df_sorted[df_sorted['customer_id'] == user_id]
#         if len(user_orders) < 2:
#             continue

#         train_orders = user_orders.iloc[:-1]
#         test_order = user_orders.iloc[-1]

#         actual_vendor = test_order['vendor_id']
#         actual_cuisine = test_order['cuisine_origin']

#         try:
#             # Generate recommendations
#             recommendations = recommender.generate_enhanced_recommendations(user_id, top_n=top_k)
#             rec_vendors = recommendations['vendor_id'].tolist()
#             rec_cuisines = recommendations['cuisine'].tolist()
#             rec_scores = recommendations['final_score'].tolist()
#             rec_ratings = recommendations['vendor_rating'].tolist()

#             # Check if recommended vendors meet quality standards (3.5+ stars)
#             quality_vendors = sum(1 for rating in rec_ratings if rating >= 3.5)

#             # Create relevance scores (1 if vendor matches, 0 otherwise)
#             relevance_scores = [1 if v == actual_vendor else 0 for v in rec_vendors]

#             # Calculate metrics
#             metrics['precision@5'].append(np.mean(relevance_scores[:5]))
#             metrics['precision@10'].append(np.mean(relevance_scores[:10]))

#             # Recall (since we have one relevant item, it's binary)
#             metrics['recall@5'].append(1 if any(relevance_scores[:5]) else 0)
#             metrics['recall@10'].append(1 if any(relevance_scores[:10]) else 0)

#             # Ranking metrics
#             metrics['ndcg@5'].append(calculate_ndcg(relevance_scores, 5))
#             metrics['ndcg@10'].append(calculate_ndcg(relevance_scores, 10))
#             metrics['mrr@5'].append(calculate_mrr(relevance_scores, 5))
#             metrics['mrr@10'].append(calculate_mrr(relevance_scores, 10))

#             # Novelty (proportion of new vendors not in user history)
#             user_history_vendors = set(train_orders['vendor_id'])
#             novel_count = sum(1 for v in rec_vendors[:10] if v not in user_history_vendors)
#             metrics['novelty'].append(novel_count / 10)

#             # Diversity (unique cuisines in recommendations)
#             unique_cuisines = len(set(rec_cuisines[:10]))
#             metrics['diversity'].append(unique_cuisines / 10)

#             # Store for coverage calculation
#             all_recommendations.extend(rec_vendors[:10])

#             # Store user analysis data for sample users
#             if i < 5:  # Store first 5 users for detailed analysis
#                 user_analysis_data.append({
#                     'user_id': user_id,
#                     'user_orders': len(user_orders),
#                     'unique_vendors': user_orders['vendor_id'].nunique(),
#                     'actual_vendor': actual_vendor,
#                     'actual_cuisine': actual_cuisine,
#                     'recommendations': recommendations.head(10).copy(),
#                     'relevance_scores': relevance_scores,
#                     'found_relevant': any(relevance_scores),
#                     'position_found': relevance_scores.index(1) + 1 if any(relevance_scores) else None,
#                     'quality_vendors': quality_vendors
#                 })

#         except Exception as e:
#             print(f"Error evaluating user {user_id}: {e}")
#             continue

#     # Calculate averages
#     avg_metrics = {k: np.mean(v) for k, v in metrics.items() if v}

#     # Coverage calculation
#     all_vendors = set(matrices['user_vendor'].columns)
#     recommended_vendors = set(all_recommendations)
#     coverage = len(recommended_vendors) / len(all_vendors) if all_vendors else 0

#     # Print comprehensive results
#     print("\n" + "="*70)
#     print("ENHANCED EVALUATION RESULTS")
#     print("="*70)

#     # Metrics with explanations
#     metric_descriptions = {
#         'precision@5': 'Accuracy in top 5 recommendations',
#         'precision@10': 'Accuracy in top 10 recommendations',
#         'recall@5': 'Ability to find relevant items in top 5',
#         'recall@10': 'Ability to find relevant items in top 10',
#         'ndcg@5': 'Ranking quality in top 5 (0-1 scale)',
#         'ndcg@10': 'Ranking quality in top 10 (0-1 scale)',
#         'mrr@5': 'How soon first relevant item appears in top 5',
#         'mrr@10': 'How soon first relevant item appears in top 10',
#         'novelty': 'Proportion of new/unseen recommendations',
#         'diversity': 'Variety of cuisines in recommendations'
#     }

#     print("\n📊 PERFORMANCE METRICS:")
#     print("-" * 50)
#     for metric, value in avg_metrics.items():
#         description = metric_descriptions.get(metric, '')
#         print(f"{metric:15}: {value:.4f} - {description}")

#     print(f"{'coverage':15}: {coverage:.4f} - Percentage of total vendors recommended")

#     # Target ranges comparison
#     print("\n🎯 TARGET RANGES COMPARISON:")
#     print("-" * 50)
#     target_ranges = {
#         'precision@10': (0.10, 0.30),
#         'recall@10': (0.20, 0.40),
#         'ndcg@10': (0.60, 0.85),
#         'mrr@10': (0.30, 0.60)
#     }

#     for metric, (low, high) in target_ranges.items():
#         if metric in avg_metrics:
#             value = avg_metrics[metric]
#             if value >= low and value <= high:
#                 status = "✅ WITHIN TARGET"
#             elif value < low:
#                 status = "📈 BELOW TARGET"
#             else:
#                 status = "🎉 ABOVE TARGET"
#             print(f"{metric:15}: {value:.4f} [Target: {low:.2f}-{high:.2f}] {status}")

#     # Sample user analysis
#     if user_analysis_data:
#         print("\n🔍 SAMPLE USER ANALYSIS:")
#         print("-" * 50)
#         for user_data in user_analysis_data[:3]:  # Show first 3 users
#             print(f"\nUser: {user_data['user_id']}")
#             print(f"  Orders: {user_data['user_orders']}, Unique Vendors: {user_data['unique_vendors']}")
#             print(f"  Actual Choice: {user_data['actual_vendor']} ({user_data['actual_cuisine']})")
#             print(f"  Found in Recs: {'Yes' if user_data['found_relevant'] else 'No'}")
#             if user_data['found_relevant']:
#                 print(f"  Position Found: #{user_data['position_found']}")
#             print(f"  Quality Vendors (3.5+ stars): {user_data['quality_vendors']}/10")

#             # Show top 3 recommendations
#             print("  Top 3 Recommendations:")
#             for idx, row in user_data['recommendations'].head(3).iterrows():
#                 print(f"    {idx+1}. {row['vendor_id']} | Score: {row['final_score']:.3f} | "
#                       f"Cuisine: {row['cuisine']} | Rating: {row['vendor_rating']:.2f}")

#     return avg_metrics, coverage, user_analysis_data

# # Run enhanced evaluation
# enhanced_metrics, enhanced_coverage, user_analysis = enhanced_evaluation(
#     enhanced_recommender, order_level_data, sample_users=100
# )

# # Print summary for quick assessment
# print("\n" + "="*70)
# print("QUICK ASSESSMENT SUMMARY")
# print("="*70)

# critical_metrics = ['precision@10', 'recall@10', 'ndcg@10', 'mrr@10']
# for metric in critical_metrics:
#     if metric in enhanced_metrics:
#         value = enhanced_metrics[metric]
#         if metric == 'ndcg@10':
#             if value >= 0.60:
#                 status = "✅ EXCELLENT"
#             elif value >= 0.30:
#                 status = "📈 GOOD"
#             else:
#                 status = "⚠️ NEEDS IMPROVEMENT"
#         elif metric in ['precision@10', 'recall@10']:
#             if value >= 0.10:
#                 status = "✅ GOOD"
#             else:
#                 status = "⚠️ NEEDS IMPROVEMENT"
#         else:  # MRR
#             if value >= 0.30:
#                 status = "✅ GOOD"
#             else:
#                 status = "⚠️ NEEDS IMPROVEMENT"

#         print(f"{metric:15}: {value:.4f} - {status}")

# Cell 7: Optimized Hybrid Recommender with Ranking Enhancement
### Purpose: This cell creates an optimized version of the hybrid recommender that focuses on improving ranking quality (NDCG) by incorporating user affinity signals and better balancing novelty with user preferences. The optimized version enhances the base recommender with ranking-specific improvements.

In [ ]:
# # =============================================
# # OPTIMIZED HYBRID RECOMMENDER (RANKING FOCUSED)
# # =============================================

# print("\nStep 7: Creating Optimized Recommender for Better Ranking...")

# class OptimizedHybridRecommender(EnhancedHybridRecommender):
#     def __init__(self, als_model, user_profiles, vendor_features, interaction_matrices):
#         super().__init__(als_model, user_profiles, vendor_features, interaction_matrices)
#         print("   Ranking-optimized recommender initialized")

#     def generate_optimized_recommendations(self, user_id, top_n=10, novelty_boost=0.1, ranking_boost=0.2):
#         """
#         Optimized version with better ranking balance and user affinity modeling
#         Focuses on improving NDCG while maintaining diversity
#         """

#         # Get base recommendations from enhanced recommender
#         base_recommendations = self.generate_enhanced_recommendations(user_id, top_n * 3, novelty_boost)

#         if len(base_recommendations) == 0:
#             return base_recommendations.head(top_n)

#         # Create a copy to avoid modifying the original
#         recommendations = base_recommendations.copy()

#         # Apply ranking optimization based on user affinity and temporal patterns
#         if user_id in self.matrices['raw_interactions']['customer_id'].values:
#             user_ordered_vendors = set(self.matrices['raw_interactions'][
#                 self.matrices['raw_interactions']['customer_id'] == user_id
#             ]['vendor_id'])

#             # Calculate user-vendor affinity scores for better ranking
#             affinity_boosts = self._calculate_user_affinity_boosts(user_id, recommendations['vendor_id'])
#             recommendations['affinity_boost'] = recommendations['vendor_id'].map(affinity_boosts)

#             # Apply ranking boost based on affinity
#             recommendations['optimized_score'] = recommendations['final_score'] * (1 + recommendations['affinity_boost'])

#             # For vendors with high affinity, ensure they rank appropriately
#             high_affinity_mask = recommendations['affinity_boost'] > 0.3
#             if high_affinity_mask.any():
#                 # Boost high-affinity vendors to top positions when appropriate
#                 max_original_score = recommendations['final_score'].max()
#                 recommendations.loc[high_affinity_mask, 'optimized_score'] = recommendations.loc[high_affinity_mask, 'optimized_score'].clip(
#                     upper=max_original_score * 1.2  # Prevent over-boosting
#                 )
#         else:
#             # For new users, use the original scores
#             recommendations['optimized_score'] = recommendations['final_score']
#             recommendations['affinity_boost'] = 0

#         # Apply temporal recency boost (prefer vendors user recently interacted with)
#         temporal_boost = self._calculate_temporal_boost(user_id, recommendations['vendor_id'])
#         recommendations['temporal_boost'] = recommendations['vendor_id'].map(temporal_boost)
#         recommendations['optimized_score'] = recommendations['optimized_score'] * (1 + recommendations['temporal_boost'])

#         # Final normalization and sorting
#         max_opt_score = recommendations['optimized_score'].max()
#         if max_opt_score > 0:
#             recommendations['optimized_score'] = recommendations['optimized_score'] / max_opt_score

#         # Sort by optimized score (CRITICAL for NDCG improvement)
#         final_recommendations = recommendations.sort_values('optimized_score', ascending=False).head(top_n)

#         print(f"   Ranking optimized: {len(final_recommendations)} recs, "
#               f"affinity range: {recommendations['affinity_boost'].min():.3f}-{recommendations['affinity_boost'].max():.3f}")

#         return final_recommendations[['vendor_id', 'optimized_score', 'cuisine', 'vendor_rating']].rename(
#             columns={'optimized_score': 'final_score'}
#         )

#     def _calculate_user_affinity_boosts(self, user_id, candidate_vendors):
#         """Calculate user-vendor affinity based on past interactions and preferences"""
#         affinity_boosts = {}

#         if user_id not in self.matrices['raw_interactions']['customer_id'].values:
#             return {vendor: 0 for vendor in candidate_vendors}

#         # Get user's interaction history
#         user_interactions = self.matrices['raw_interactions'][
#             self.matrices['raw_interactions']['customer_id'] == user_id
#         ]

#         # Get user's cuisine preferences from profile
#         user_cuisine_prefs = {}
#         if user_id in self.user_profiles['customer_id'].values:
#             user_profile = self.user_profiles[self.user_profiles['customer_id'] == user_id].iloc[0]
#             # Extract cuisine preferences (columns that are cuisine names)
#             cuisine_columns = [col for col in self.user_profiles.columns if col not in
#                              ['customer_id', 'user_cluster'] and col in user_profile.index]
#             for cuisine in cuisine_columns:
#                 if cuisine in user_profile:
#                     user_cuisine_prefs[cuisine] = user_profile[cuisine]

#         for vendor_id in candidate_vendors:
#             boost = 0

#             # Check if user has ordered from this vendor before
#             vendor_interactions = user_interactions[user_interactions['vendor_id'] == vendor_id]
#             if len(vendor_interactions) > 0:
#                 # Calculate affinity based on order frequency and ratings
#                 order_count = vendor_interactions['order_id'].iloc[0]
#                 avg_rating = vendor_interactions['avg_product_rating'].iloc[0]

#                 # Strong boost for frequently ordered, highly rated vendors
#                 frequency_boost = min(0.4, order_count * 0.15)  # Cap at 0.4
#                 rating_boost = min(0.3, (avg_rating - 3.5) * 0.2) if avg_rating > 3.5 else 0

#                 boost = frequency_boost + rating_boost

#             # Check cuisine preference match
#             if vendor_id in self.vendor_features.index:
#                 vendor_cuisine = self.vendor_features.loc[vendor_id, 'cuisine_origin']
#                 if vendor_cuisine in user_cuisine_prefs and user_cuisine_prefs[vendor_cuisine] > 0.1:
#                     cuisine_boost = user_cuisine_prefs[vendor_cuisine] * 0.2  # Up to 0.2 boost
#                     boost += cuisine_boost

#             affinity_boosts[vendor_id] = min(boost, 0.6)  # Cap total boost at 0.6

#         return affinity_boosts

#     def _calculate_temporal_boost(self, user_id, candidate_vendors):
#         """Calculate temporal boosts based on recency and patterns"""
#         temporal_boosts = {}

#         if user_id not in self.matrices['raw_interactions']['customer_id'].values:
#             return {vendor: 0 for vendor in candidate_vendors}

#         # Get user's recent interactions
#         user_interactions = self.matrices['raw_interactions'][
#             self.matrices['raw_interactions']['customer_id'] == user_id
#         ]

#         if len(user_interactions) == 0:
#             return {vendor: 0 for vendor in candidate_vendors}

#         max_order_day = user_interactions['order_day'].max()

#         for vendor_id in candidate_vendors:
#             boost = 0

#             # Check recency for this vendor
#             vendor_recency = user_interactions[user_interactions['vendor_id'] == vendor_id]
#             if len(vendor_recency) > 0:
#                 last_order_day = vendor_recency['order_day'].max()
#                 days_ago = max_order_day - last_order_day

#                 # Boost for recently ordered vendors (within last 30 days)
#                 if days_ago <= 30:
#                     recency_boost = 0.2 * (1 - (days_ago / 30))
#                     boost += recency_boost

#             temporal_boosts[vendor_id] = boost

#         return temporal_boosts

# # Initialize optimized recommender
# optimized_recommender = OptimizedHybridRecommender(als_model, user_profiles_with_clusters, vendor_features, matrices)
# print("✅ Optimized hybrid recommender ready!")

# def run_optimized_evaluation():
#     """Run comprehensive evaluation with optimized recommender"""
#     print("Running OPTIMIZED evaluation...")

#     # Use temporal split but ensure minimum test interactions
#     df_sorted = order_level_data.sort_values('order_day')

#     # Only evaluate users with sufficient history
#     user_order_counts = df_sorted['customer_id'].value_counts()
#     eligible_users = user_order_counts[user_order_counts >= 3].index
#     test_users = np.random.choice(eligible_users, min(100, len(eligible_users)), replace=False)

#     print(f"Evaluating {len(test_users)} users with sufficient history...")

#     # Enhanced metrics including MRR
#     metrics = {
#         'precision@5': [], 'precision@10': [],
#         'recall@5': [], 'recall@10': [],
#         'ndcg@5': [], 'ndcg@10': [],
#         'mrr@5': [], 'mrr@10': [],
#         'novelty': [], 'diversity': []
#     }

#     detailed_results = []

#     for i, user_id in enumerate(test_users):
#         if i % 20 == 0:
#             print(f"Progress: {i}/{len(test_users)}")

#         # Get user's last order as test
#         user_orders = df_sorted[df_sorted['customer_id'] == user_id]
#         if len(user_orders) < 2:
#             continue

#         train_orders = user_orders.iloc[:-1]
#         test_order = user_orders.iloc[-1]

#         actual_vendor = test_order['vendor_id']
#         actual_cuisine = test_order['cuisine_origin']

#         try:
#             recommendations = optimized_recommender.generate_optimized_recommendations(user_id, top_n=10)

#             if len(recommendations) == 0:
#                 continue

#             rec_vendors = recommendations['vendor_id'].tolist()
#             rec_cuisines = recommendations['cuisine'].tolist()
#             rec_scores = recommendations['final_score'].tolist()

#             # Create relevance scores
#             relevance_scores = [1 if v == actual_vendor else 0 for v in rec_vendors]

#             # Calculate all metrics
#             metrics['precision@5'].append(np.mean(relevance_scores[:5]))
#             metrics['precision@10'].append(np.mean(relevance_scores[:10]))

#             metrics['recall@5'].append(1 if any(relevance_scores[:5]) else 0)
#             metrics['recall@10'].append(1 if any(relevance_scores[:10]) else 0)

#             metrics['ndcg@5'].append(calculate_ndcg(relevance_scores, 5))
#             metrics['ndcg@10'].append(calculate_ndcg(relevance_scores, 10))

#             metrics['mrr@5'].append(calculate_mrr(relevance_scores, 5))
#             metrics['mrr@10'].append(calculate_mrr(relevance_scores, 10))

#             # Novelty
#             user_history_vendors = set(train_orders['vendor_id'])
#             novel_count = sum(1 for v in rec_vendors[:10] if v not in user_history_vendors)
#             metrics['novelty'].append(novel_count / 10)

#             # Diversity
#             unique_cuisines = len(set(rec_cuisines[:10]))
#             metrics['diversity'].append(unique_cuisines / 10)

#             # Store detailed results for analysis
#             if any(relevance_scores):
#                 position_found = relevance_scores.index(1) + 1
#                 detailed_results.append({
#                     'user_id': user_id,
#                     'position_found': position_found,
#                     'score_at_position': rec_scores[position_found-1] if position_found <= len(rec_scores) else 0
#                 })

#         except Exception as e:
#             continue

#     # Calculate averages
#     avg_metrics = {}
#     for k, v in metrics.items():
#         if v:
#             avg_metrics[k] = np.mean(v)
#         else:
#             avg_metrics[k] = 0

#     # Print comprehensive results
#     print("\n" + "="*70)
#     print("OPTIMIZED EVALUATION RESULTS")
#     print("="*70)

#     # Metrics with target comparisons
#     target_ranges = {
#         'precision@10': (0.10, 0.30),
#         'recall@10': (0.20, 0.40),
#         'ndcg@10': (0.60, 0.85),
#         'mrr@10': (0.30, 0.60)
#     }

#     print("\n📊 PERFORMANCE METRICS:")
#     print("-" * 50)
#     for metric, value in avg_metrics.items():
#         if metric in target_ranges:
#             low, high = target_ranges[metric]
#             if value >= low and value <= high:
#                 status = "✅ WITHIN TARGET"
#             elif value < low:
#                 status = "📈 BELOW TARGET"
#             else:
#                 status = "🎉 ABOVE TARGET"
#             print(f"{metric:15}: {value:.4f} [Target: {low:.2f}-{high:.2f}] {status}")
#         else:
#             print(f"{metric:15}: {value:.4f}")

#     # Ranking analysis
#     if detailed_results:
#         positions = [r['position_found'] for r in detailed_results]
#         avg_position = np.mean(positions)
#         print(f"\n🎯 RANKING ANALYSIS:")
#         print(f"   Average position of relevant items: {avg_position:.2f}")
#         print(f"   Top-1 accuracy: {sum(1 for p in positions if p == 1) / len(positions):.3f}")
#         print(f"   Top-3 accuracy: {sum(1 for p in positions if p <= 3) / len(positions):.3f}")

#     return avg_metrics

# # Run optimized evaluation
# optimized_metrics = run_optimized_evaluation()

# # Compare with previous results if available
# try:
#     print("\n" + "="*70)
#     print("COMPARISON: ENHANCED vs OPTIMIZED")
#     print("="*70)

#     comparison_metrics = ['precision@10', 'recall@10', 'ndcg@10', 'mrr@10']
#     for metric in comparison_metrics:
#         if metric in enhanced_metrics and metric in optimized_metrics:
#             enhanced_val = enhanced_metrics[metric]
#             optimized_val = optimized_metrics[metric]
#             improvement = optimized_val - enhanced_val
#             change = "↑" if improvement > 0 else "↓"
#             print(f"{metric:15}: {enhanced_val:.4f} → {optimized_val:.4f} {change}{abs(improvement):.4f}")
# except NameError:
#     print("Note: Enhanced metrics not available for comparison")


Step 7: Creating Optimized Recommender for Better Ranking...
   Aligned vendor features: 1326 vendors
✅ Recommender initialized with 1326 quality vendors (3.5+ stars)
   Ranking-optimized recommender initialized
✅ Optimized hybrid recommender ready!
Running OPTIMIZED evaluation...
Evaluating 100 users with sufficient history...
Progress: 0/100
   User c712101860: CF=0.7, CB=0.2, POP=0.1


   Generated 30 recommendations, score range: 0.448-1.374
   Ranking optimized: 10 recs, affinity range: 0.000-0.574
   User 0ec52a71f0: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, score range: 0.555-1.397
   Ranking optimized: 10 recs, affinity range: 0.000-0.478
   User 0370caccb1: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, score range: 0.476-1.396
   Ranking optimized: 10 recs, affinity range: 0.000-0.073
   User 3517a17ed0: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, score range: 0.414-1.188
   Ranking optimized: 10 recs, affinity range: 0.000-0.312
   User cf23f665af: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, score range: 0.404-1.296
   Ranking optimized: 10 recs, affinity range: 0.000-0.506
   User 903444c37e: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, score range: 0.435-1.163
   Ranking optimized: 10 recs, affinity range: 0.000-0.568
   User 65c5277cd8: CF=0.7, CB=0.2, POP=0.1
   Generated 30 recommendations, s

In [ ]:
# =============================================
# FINAL PRODUCTION RECOMMENDATION STRATEGY
# =============================================

print("\nStep 8: Final Production Recommendation Strategy...")

def get_final_recommendations(user_id, top_n=10):
    """
    Final production recommendation strategy:
    - Use optimized for most users (better ranking)
    - Fallback to enhanced for cold-start users
    - Ultimate fallback to popularity-based
    """
    try:
        # Try optimized first (better ranking)
        recommendations = optimized_recommender.generate_optimized_recommendations(user_id, top_n)

        # If no recommendations or poor scores, use enhanced
        if len(recommendations) == 0 or recommendations['final_score'].max() < 0.1:
            print(f"Using enhanced fallback for user {user_id}")
            recommendations = enhanced_recommender.generate_enhanced_recommendations(user_id, top_n)

        return recommendations

    except Exception as e:
        # Ultimate fallback to popularity-based
        print(f"Using ultimate fallback for user {user_id}: {e}")
        return enhanced_recommender._get_fallback_recommendations(top_n)

def analyze_recommendation_quality(user_id, recommendations):
    """Analyze the quality of recommendations for a user"""
    if len(recommendations) == 0:
        print("No recommendations generated")
        return

    # Get user history
    user_history = order_level_data[order_level_data['customer_id'] == user_id]
    user_vendors = set(user_history['vendor_id'])
    user_cuisines = set(user_history['cuisine_origin'])

    print(f"\n📊 Recommendation Quality Analysis for User {user_id}:")
    print(f"   User History: {len(user_history)} orders, {len(user_vendors)} unique vendors")
    print(f"   User Cuisines: {list(user_cuisines)}")

    # Calculate metrics
    ordered_before = sum(1 for vendor in recommendations['vendor_id'] if vendor in user_vendors)
    new_vendors = len(recommendations) - ordered_before
    cuisine_diversity = len(set(recommendations['cuisine']))

    print(f"   Recommendations: {len(recommendations)} total")
    print(f"   - Ordered before: {ordered_before}")
    print(f"   - New vendors: {new_vendors}")
    print(f"   - Cuisine diversity: {cuisine_diversity} unique cuisines")
    print(f"   - Score range: {recommendations['final_score'].min():.3f} to {recommendations['final_score'].max():.3f}")
    print(f"   - Average vendor rating: {recommendations['vendor_rating'].mean():.2f}")

# Test final strategy with multiple users
print("Testing final production recommendation strategy...")
print("="*70)

# Test with different types of users
test_users = [
    order_level_data['customer_id'].iloc[0],  # Active user
    order_level_data['customer_id'].iloc[100], # Medium user
    order_level_data['customer_id'].iloc[500]  # Another user
]

for i, user_id in enumerate(test_users, 1):
    print(f"\n🧪 TEST {i}: User {user_id}")
    print("-" * 50)

    try:
        final_recs = get_final_recommendations(user_id, 5)

        if len(final_recs) > 0:
            print(f"Final recommendations:")
            for j, (_, rec) in enumerate(final_recs.iterrows(), 1):
                is_ordered = "✓" if rec['vendor_id'] in set(order_level_data[
                    order_level_data['customer_id'] == user_id
                ]['vendor_id']) else " "

                print(f"  {j}. {rec['vendor_id']} | Score: {rec['final_score']:.3f} | "
                      f"Cuisine: {rec['cuisine']} | Rating: {rec['vendor_rating']:.2f} | "
                      f"Ordered: [{is_ordered}]")

            # Analyze quality
            analyze_recommendation_quality(user_id, final_recs)
        else:
            print("  No recommendations generated")

    except Exception as e:
        print(f"  Error: {e}")

print("\n" + "="*70)
print("🎯 PRODUCTION SYSTEM READY!")
print("="*70)
print("Use 'get_final_recommendations(user_id, top_n)' for production recommendations")
print("\nSystem Features:")
print("✅ Hybrid collaborative + content-based filtering")
print("✅ Optimized ranking with novelty-diversity balance")
print("✅ Multiple fallback strategies for robustness")
print("✅ Comprehensive evaluation metrics")
print("✅ Production-ready recommendation function")


Step 8: Final Production Recommendation Strategy...
Testing final production recommendation strategy...

🧪 TEST 1: User 2e7276ad3a
--------------------------------------------------
Using ultimate fallback for user 2e7276ad3a: 'OptimizedHybridRecommender' object has no attribute 'generate_optimized_recommendations'
  Error: name 'enhanced_recommender' is not defined

🧪 TEST 2: User fec4d1f339
--------------------------------------------------
Using ultimate fallback for user fec4d1f339: 'OptimizedHybridRecommender' object has no attribute 'generate_optimized_recommendations'
  Error: name 'enhanced_recommender' is not defined

🧪 TEST 3: User ac01e97093
--------------------------------------------------
Using ultimate fallback for user ac01e97093: 'OptimizedHybridRecommender' object has no attribute 'generate_optimized_recommendations'
  Error: name 'enhanced_recommender' is not defined

🎯 PRODUCTION SYSTEM READY!
Use 'get_final_recommendations(user_id, top_n)' for production recommend

In [28]:
get_final_recommendations('2e7276ad3a', top_n=10)  # Example user_id

Using ultimate fallback for user 2e7276ad3a: 'OptimizedHybridRecommender' object has no attribute 'generate_optimized_recommendations'


NameError: name 'enhanced_recommender' is not defined

In [29]:
get_final_recommendations('2e7276ad3a', top_n=10)  # Example user_id

Using ultimate fallback for user 2e7276ad3a: 'OptimizedHybridRecommender' object has no attribute 'generate_optimized_recommendations'


NameError: name 'enhanced_recommender' is not defined

### Different Selection Strategy

In [ ]:
# =============================================
# BETTER USER SELECTION STRATEGIES
# =============================================

def select_test_users(df, strategy="diverse", n_users=5):
    """
    Select users for testing recommendations based on different strategies
    """

    # Calculate user order counts
    user_order_counts = df.groupby('customer_id').size()

    if strategy == "diverse":
        # Select users with different order frequencies
        return {
            'new_user': user_order_counts[user_order_counts == 1].index[0],      # 1 order
            'medium_user': user_order_counts[user_order_counts.between(2, 5)].index[0],  # 2-5 orders
            'power_user': user_order_counts[user_order_counts > 10].index[0],    # 10+ orders
            'random_user': df['customer_id'].sample(1).iloc[0],                  # Random
            'first_user': df['customer_id'].iloc[0]                              # Your current method
        }

    elif strategy == "power_users":
        # Users with most orders (best for testing CF)
        return user_order_counts.nlargest(n_users).index.tolist()

    elif strategy == "new_users":
        # Users with few orders (tests CB fallback)
        return user_order_counts.nsmallest(n_users).index.tolist()

    elif strategy == "coverage":
        # Users who ordered different cuisines
        user_cuisine_diversity = df.groupby('customer_id')['cuisine_origin'].nunique()
        return user_cuisine_diversity.nlargest(n_users).index.tolist()

# Test with diverse users
test_users = select_test_users(df, strategy="diverse")

print("🧪 TESTING WITH DIVERSE USERS:")
for user_type, user_id in test_users.items():
    user_orders = len(df[df['customer_id'] == user_id])
    user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

    print(f"{user_type}: {user_id} ({user_orders} orders, {user_cuisines} cuisines)")

    # Generate recommendations for this user
    recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)
    top_cuisine = vendor_features.loc[recommendations.iloc[0]['vendor_id'], 'cuisine_origin']

    print(f"  Top recommendation: {top_cuisine}")
    print(f"  CF score: {recommendations.iloc[0]['cf_score']:.6f}")
    print(f"  CB score: {recommendations.iloc[0]['cb_score']:.3f}")
    print()

🧪 TESTING WITH DIVERSE USERS:
new_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000

medium_user: 008ce71183 (3 orders, 2 cuisines)
  Top recommendation: american
  CF score: 0.045964
  CB score: 0.679

power_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

random_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

first_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000



In [13]:
# Let's investigate the power user more closely
power_user_id = '046958a898'
power_user_orders = df[df['customer_id'] == power_user_id]

print(f"🔍 ANALYZING POWER USER: {power_user_id}")
print("=" * 50)

# Check their actual order history
print("📋 ORDER HISTORY:")
cuisine_history = power_user_orders['cuisine_origin'].value_counts()
for cuisine, count in cuisine_history.items():
    print(f"  - {cuisine}: {count} orders")

# Check their preference profile
if power_user_id in user_profiles['customer_id'].values:
    user_profile = user_profiles[user_profiles['customer_id'] == power_user_id].iloc[0]
    print(f"\n🎯 PREFERENCE PROFILE:")
    # Get top 3 cuisine preferences
    cuisine_cols = [col for col in user_profile.index if col in df['cuisine_origin'].unique()]
    cuisine_prefs = user_profile[cuisine_cols].astype(float)
    top_preferences = cuisine_prefs.nlargest(3)
    for cuisine, score in top_preferences.items():
        print(f"  - {cuisine}: {score:.3f}")

# Check why snacks was recommended
snacks_recommendation = recommendations[recommendations['vendor_id'] == recommendations.iloc[0]['vendor_id']]
print(f"\n🤔 WHY SNACKS RECOMMENDATION?")
print(f"Vendor: {snacks_recommendation['vendor_id'].iloc[0]}")
print(f"CF Score: {snacks_recommendation['cf_score'].iloc[0]:.6f}")
print(f"CB Score: {snacks_recommendation['cb_score'].iloc[0]:.3f}")

🔍 ANALYZING POWER USER: 046958a898
📋 ORDER HISTORY:
  - snacks: 6 orders
  - american: 3 orders
  - italian: 2 orders
  - singaporean: 1 orders

🎯 PREFERENCE PROFILE:
  - snacks: 0.487
  - american: 0.249
  - italian: 0.188

🤔 WHY SNACKS RECOMMENDATION?
Vendor: 90cca11f
CF Score: 0.000055
CB Score: 1.000


In [ ]:
# =============================================
# COMPREHENSIVE MODEL EVALUATION
# =============================================

def evaluate_model_performance(df, als_model, user_profiles, vendor_features, n_test_users=10):
    """
    Evaluate recommendation quality across different user types
    """

    user_order_counts = df.groupby('customer_id').size()

    print("📊 MODEL PERFORMANCE EVALUATION")
    print("=" * 50)

    # Test different user segments
    segments = {
        'New Users (1 order)': user_order_counts[user_order_counts == 1].index[:3],
        'Medium Users (2-5 orders)': user_order_counts[user_order_counts.between(2, 5)].index[:3],
        'Power Users (6+ orders)': user_order_counts[user_order_counts >= 6].index[:3],
        'Diverse Cuisine Users': df.groupby('customer_id')['cuisine_origin'].nunique().nlargest(3).index
    }

    for segment_name, user_ids in segments.items():
        print(f"\n🎯 {segment_name}:")
        print("-" * 30)

        for user_id in user_ids:
            if user_id in user_vendor_matrix.index:
                recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)

                # Basic user stats
                user_orders = len(df[df['customer_id'] == user_id])
                user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

                # Recommendation stats
                top_rec = recommendations.iloc[0]
                top_cuisine = vendor_features.loc[top_rec['vendor_id'], 'cuisine_origin']

                print(f"  User {user_id[:8]}...: {user_orders} orders, {user_cuisines} cuisines")
                print(f"    → Top rec: {top_cuisine} (CF: {top_rec['cf_score']:.4f}, CB: {top_rec['cb_score']:.2f})")

# Run comprehensive evaluation
evaluate_model_performance(df, als_model, user_profiles, vendor_features)

📊 MODEL PERFORMANCE EVALUATION

🎯 New Users (1 order):
------------------------------
  User 008ab40a...: 1 orders, 1 cuisines
    → Top rec: chinese (CF: 0.0001, CB: 1.00)
  User 00ba08ba...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0342, CB: 1.00)
  User 00e9c13b...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0098, CB: 1.00)

🎯 Medium Users (2-5 orders):
------------------------------
  User 008ce711...: 3 orders, 2 cuisines
    → Top rec: american (CF: 0.0460, CB: 0.68)
  User 00c41737...: 2 orders, 2 cuisines
    → Top rec: chinese (CF: 0.0267, CB: 0.49)
  User 01058d1d...: 2 orders, 1 cuisines
    → Top rec: snacks (CF: 0.1304, CB: 1.00)

🎯 Power Users (6+ orders):
------------------------------
  User 01907d62...: 9 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0977, CB: 0.65)
  User 027862ef...: 6 orders, 3 cuisines
    → Top rec: chinese (CF: 0.0216, CB: 0.66)
  User 028a29c2...: 6 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0577, CB: 0.48)

🎯 Diverse 